In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [13]:
mol_str = """
0 1
o
h 1 1.0
h 1 1.0 2 104.0
symmetry c1
"""

options_dict = {'basis': '6-31g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)

cavity_options = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'davidson_roots' : 20,
    'nact_orbs' : 0,
    'nact_els' : 0
}

psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [14]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.0])
#lambda_vector = np.array([0.5, 0.5, 0.5])

n_act_orb=5
#number of active electrons
n_act_el=4
# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)


0

Start SCF iterations:

Canonical RHF One-electron energy = -122.2743570184695727
CQED-RHF One-electron energy      = -122.2743570184695727
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -75.9800163449004344   dE = -7.59800E+01   dRMS = 5.74391E-13
SCF Iteration   2: Energy = -75.9800163449003350   dE =  9.94760E-14   dRMS = 1.52547E-13
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -75.98001634 hartree
Psi4  SCF energy: -75.98001634 hartree


(130, 40)
iteration 1 dimension 40
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
root residual norm Eigenvalue Convergence
1 1.4701794490121018e-12 -75.98001634490073 Y
2 1.4353234676174677e-12 -75.98001634490038 Y
3 0.10896415703178265 -75.67931419776276 N
4 0.10896415703178335 -75.67931419776241 N
5 0.08493271795655417 -75.64621847406018 N
6 0.08493271795655594 -75.64621847406009 N
7 0.14314896629510218 -75.603629245

In [15]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [22]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


checking to see if the Davidson Roots Agree!
[-75.98001634 -75.98001634 -75.68870627 -75.68870627 -75.65206912
 -75.65206912 -75.62786334 -75.62786334 -75.61052301 -75.61052301
 -75.58511723 -75.58511723 -75.56878651 -75.56878651]
[-75.98001634 -75.98001634 -75.68870627 -75.68870627 -75.65206912
 -75.65206912 -75.62786334 -75.62786334 -75.61052301 -75.61052301
 -75.58511723 -75.58511723 -75.56878651 -75.56878651]
True


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)